# Taller Ecuaciones Nodales

In [2]:
import numpy as np
from numpy import pi

In [5]:
zl = 0.0021+0.005j
Bc = 0.0008j

v1, v2, v3, v4, v5 = 1.03, 0.97, 1.01, 0.95, 0.985
v = np.array([v1, v2, v3, v4, v5])
ang = np.array([0, -6.3, -2.2, -4.2, -6.2])*pi/180
ang1, ang2, ang3, ang4, ang5 = ang[0], ang[1], ang[2], ang[3], ang[4]

e = v*np.exp(1j*ang)
print(e)

[1.03      +0.j         0.96414213-0.10644228j 1.00925555-0.03877169j
 0.94744875-0.06957629j 0.9792387 -0.10637937j]


## Punto 1

In [15]:
y12, y13, y14, y15 = 1/(zl*145), 1/(zl*160), 0, 1/(zl*80)

y21, y23, y24, y25 = y12, 0, 1/(zl*155), 0

y31, y32, y34, y35 = y13, y23, 1/(zl*170), 0

y41, y42, y43, y45 = y14, y24, y34, 1/(zl*122)

y51, y52, y53, y54 = y15, y25, y35, y45

B1 = 1/2*Bc*(160+80+145)
B2 = 1/2*Bc*(155+145)
B3 = 1/2*Bc*(160+170)
B4 = 1/2*Bc*(170+122+155)
B5 = 1/2*Bc*(80+122)
B = [B1,B2,B3,B4,B5]

Y = np.array([[B1+y12+y13+y14+y15, -y12, -y13, -y14, -y15],
              [-y21, B2+y21+y23+y24+y25, -y23, -y24, -y25],
              [-y31, -y32, B3+y31+y32+y34+y35, -y34, -y35],
              [-y41, -y42, -y43, B4+y41+y42+y43+y45, -y45],
              [-y51, -y52, -y53, -y54, B5+y51+y52+y53+y54]])

print("Matriz de admitancia")
print(Y)
print()

Matriz de admitancia
[[ 1.83127367-4.2061754j  -0.49244334+1.17248414j -0.44627678+1.06256375j
   0.        +0.j         -0.89255355+2.12512751j]
 [-0.49244334+1.17248414j  0.95311614-2.14932415j  0.        +0.j
  -0.4606728 +1.09684j     0.        +0.j        ]
 [-0.44627678+1.06256375j  0.        +0.j          0.86630198-1.93062376j
  -0.4200252 +1.00006j     0.        +0.j        ]
 [ 0.        +0.j         -0.4606728 +1.09684j    -0.4200252 +1.00006j
   1.46597902-3.31162624j -0.58528102+1.39352623j]
 [-0.89255355+2.12512751j  0.        +0.j          0.        +0.j
  -0.58528102+1.39352623j  1.47783457-3.43785374j]]



## Punto 2

In [16]:
Z = np.linalg.inv(Y)

print("Matriz de impedancia")
print(Z)
print()

Matriz de impedancia
[[ 0.06396085-1.35604078j -0.01391001-1.53480788j -0.02102501-1.55028071j
  -0.03590804-1.58370993j  0.0125605 -1.47510378j]
 [-0.01391001-1.53480788j  0.12429621-1.22117085j -0.05574281-1.62569161j
  -0.01732672-1.54247979j -0.02867991-1.56822284j]
 [-0.02102501-1.55028071j -0.05574281-1.62569161j  0.1299554 -1.210372j
  -0.02417134-1.55729124j -0.03595656-1.58367572j]
 [-0.03590804-1.58370993j -0.01732672-1.54247979j -0.02417134-1.55729124j
   0.06439672-1.35563236j -0.00884247-1.52303942j]
 [ 0.0125605 -1.47510378j -0.02867991-1.56822284j -0.03595656-1.58367572j
  -0.00884247-1.52303942j  0.09696252-1.27825042j]]



## Punto 3

In [32]:
IN = Y @ e
#Corrientes netas inyectadas
print("Corrientes netas inyectadas (magnitud, ángulo)")
for i in IN:
    print("{:.4} ∠ {:.5}°".format(abs(i),np.angle(i)*180/pi))
print()

Corrientes netas inyectadas (magnitud, ángulo)
0.4929 ∠ 13.629°
0.2061 ∠ 149.3°
0.08982 ∠ 82.689°
0.3201 ∠ 86.221°
0.2966 ∠ 174.93°



## Punto 4

In [45]:
def Pn(v,Y,i):
    return v[i-1]*sum([abs(Y[i-1,k])*v[k]*cos(np.angle(Y[i-1,k])-ang[i-1]+ang[k]) for k in range(len(v))])

def Qn(v,Y,i):
    return -v[i-1]*sum([abs(Y[i-1,k])*v[k]*sin(np.angle(Y[i-1,k])-ang[i-1]+ang[k]) for k in range(len(v))])

P = np.array([Pn(v,Y,i) for i in range(1,6)])
Q = np.array([Qn(v,Y,i) for i in range(1,6)])

SN = P+Q*1j

for i in SN:
    print("{:.4f}".format(i))
print()

0.4934-0.1196j
-0.1820-0.0826j
0.0081-0.0904j
-0.0022-0.3041j
-0.2921+0.0058j



## Punto 5

In [64]:
I_ik = np.array([[(e[i]-e[k])*-Y[i,k] for k in range(5)] for i in range(5)])

I_t = e * B

S_ik = (e * np.conjugate(I_ik.T)).T

S_t = e * np.conjugate(I_t)

print("Potencias de transferencia")
print()
for i in range(1,6):
    print("S_"+str(i)+str(0)+":", round(S_t[i-1].real,4) + round(S_t[i-1].imag,4)*1j)
    for k in range(1,6):
        if i!=k and (np.abs(S_ik[i-1,k-1]) > 0.00001):
            print("S_"+str(i)+str(k)+":", round(S_ik[i-1,k-1].real,4) + round(S_ik[i-1,k-1].imag,4)*1j)

    print()

Potencias de transferencia

S_10: -0.1634j
S_12: (0.162+0.0255j)
S_13: (0.052+0.0049j)
S_15: (0.2795+0.0133j)

S_20: -0.1129j
S_21: (-0.1542-0.0072j)
S_24: (-0.0278+0.0375j)

S_30: -0.1347j
S_31: (-0.0511-0.0028j)
S_34: (0.0592+0.0471j)

S_40: -0.1614j
S_42: (0.0286-0.0357j)
S_43: (-0.0572-0.0424j)
S_45: (0.0264-0.0647j)

S_50: -0.0784j
S_51: (-0.2671+0.0162j)
S_54: (-0.025+0.0679j)



## Punto 6

In [7]:
print("Corrientes de transferencia")
print()
for i in range(1,6):
    print("I_"+str(i)+str(0)+":", round(np.abs(I_t[i-1]),4), "∠", str(round(np.angle(I_t[i-1])*180/pi,4))+"°")
    for k in range(1,6):
        if i!=k and np.abs(I_ik[i-1,k-1]) > 0.00001:
            print("I_"+str(i)+str(k)+":", round(np.abs(I_ik[i-1,k-1]),4), "∠", str(round(np.angle(I_ik[i-1,k-1])*180/pi,4))+"°")
    print()

Corrientes de transferencia

I_10: 0.1586 ∠ 90.0°
I_12: 0.1592 ∠ -8.9635°
I_13: 0.0507 ∠ -5.3662°
I_15: 0.2717 ∠ -2.7268°

I_20: 0.1164 ∠ 83.7°
I_21: 0.1592 ∠ 171.0365°
I_24: 0.0481 ∠ -132.856°

I_30: 0.1333 ∠ 87.8°
I_31: 0.0507 ∠ 174.6338°
I_34: 0.0749 ∠ -40.7259°

I_40: 0.1699 ∠ 85.8°
I_42: 0.0481 ∠ 47.144°
I_43: 0.0749 ∠ 139.2741°
I_45: 0.0735 ∠ 63.6024°

I_50: 0.0796 ∠ 83.8°
I_51: 0.2717 ∠ 177.2732°
I_54: 0.0735 ∠ -116.3976°



## Punto 7

In [66]:
SL1 = np.dot(e,np.conjugate(IN))
print("Potencia de pérdidas a)")
print(SL1)
print()

SL2 = sum(SN)
print("Potencia de pérdidas b)")
print(SL2)
print()

SL3 = sum(S_t) + sum([sum([S_ik[i,k]+S_ik[k,i] for k in range(i+1,5)]) \
                      for i in range(4)])
print("Potencia de pérdidas c)")
print(SL3)
print()

SL4 = sum([sum([abs(I_ik[i,k])**2*-1/(Y[i,k]) if (i!=k and Y[i,k]!=0) else 0 \
                for k in range(i+1,5)]) for i in range(4)]) \
    + sum([abs(I_t[i])**2/B[i] for i in range(5)])
print("Potencia de pérdidas d)")
print(SL4)

Potencia de pérdidas a)
(0.02512042981291686-0.5908904328263873j)

Potencia de pérdidas b)
(0.02512042981292051-0.590890432826387j)

Potencia de pérdidas c)
(0.02512042981291707-0.590890432826388j)

Potencia de pérdidas d)
(0.025120429812917087-0.5908904328263879j)
